In [38]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,KFold

import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


In [39]:
dataset=pd.read_csv('/content/adult.csv')

In [40]:
dataset.isnull().sum()

,0
age,0
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,0
relationship,0
race,0
sex,0


In [41]:
dataset.head(4)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K


In [42]:
dataset['sex']=dataset['sex'].map({'Male':1,'Female':0})
dataset['marital.status']=dataset['marital.status'].replace(['Never-married','Divorced','Separated','Widowed'],0)
dataset['marital.status']=dataset['marital.status'].replace(['Married-civ-spouse','Married-AF-spouse','Married-spouse-absent'],1)
print(dataset['marital.status'].replace(['Never-married','Divorced','Separated','Widowed'],0))
print(dataset['marital.status'].replace(['Married-civ-spouse','Married-AF-spouse','Married-spouse-absent'],1))


0        0
1        0
2        0
3        0
4        0
        ..
32556    0
32557    1
32558    1
32559    0
32560    0
Name: marital.status, Length: 32561, dtype: int64
0        0
1        0
2        0
3        0
4        0
        ..
32556    0
32557    1
32558    1
32559    0
32560    0
Name: marital.status, Length: 32561, dtype: int64


In [43]:
dataset.drop(labels=["workclass","education","occupation","relationship","race","native.country"],axis=1,inplace=True)

In [44]:
dataset.head()

,age,fnlwgt,education.num,marital.status,sex,capital.gain,capital.loss,hours.per.week,income
0,90,77053,9,0,0,0,4356,40,<=50K
1,82,132870,9,0,0,0,4356,18,<=50K
2,66,186061,10,0,0,0,4356,40,<=50K
3,54,140359,4,0,0,0,3900,40,<=50K
4,41,264663,10,0,0,0,3900,40,<=50K


In [48]:
y=dataset['income']
x=dataset.drop(labels=["income"],axis=1)
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [53]:
models=[]
models.append({"LR",LogisticRegression()})


In [54]:
results=dict()

for name,model in models:
  kfold=KFold(n_splits=10,random_state=7,shuffle=True)
  cv_results=cross_val_score(model,X_train,Y_train,cv=kfold,scoring='accuracy')
  results[name]=(cv_results.mean(),cv_results.std())



name result.mean  result.std
LR (0.7974896143833051, 0.007657528365294945)


In [56]:
from sklearn.model_selection import GroupKFold

# Add a 'group' column for GroupKFold example (here just an example column)
X_train['group'] = np.random.randint(0, 10, size=len(X_train))

# Initialize GroupKFold cross-validator
group_kfold = GroupKFold(n_splits=10)

# Evaluate model using GroupKFold Cross-Validation
cv_results_group = cross_val_score(LogisticRegression(), X_train, Y_train, cv=group_kfold, groups=X_train['group'], scoring='accuracy')
results["GroupKFold"] = (cv_results_group.mean(), cv_results_group.std())


In [57]:
from sklearn.model_selection import TimeSeriesSplit

# Initialize TimeSeriesSplit cross-validator
tscv = TimeSeriesSplit(n_splits=10)

# Evaluate model using TimeSeriesSplit Cross-Validation
cv_results_ts = cross_val_score(LogisticRegression(), X_train, Y_train, cv=tscv, scoring='accuracy')
results["TimeSeriesSplit"] = (cv_results_ts.mean(), cv_results_ts.std())


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [58]:

print()
print("name result.mean  result.std")

for key,value in results.items():
  print(key,value)


name result.mean  result.std
LR (0.7974896143833051, 0.007657528365294945)
GroupKFold (0.797609860128714, 0.009984159741670239)
TimeSeriesSplit (0.7969594594594595, 0.006628318972376151)
